# Extraction des Features pour les Données GEMStat

## Objectif

Extraire les mêmes features que pour le training original pour les 1415 nouvelles observations GEMStat.

## Stratégie

- **25 sites uniques** : Extraire les features fixes (DEM, SoilGrids, WorldCover, Water Type) une fois par site
- **1415 observations** : Extraire les features temporelles (Landsat, TerraClimate) par observation

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import requests
import time
import os

import rasterio
from rasterio.windows import from_bounds
from rasterio.crs import CRS
from rasterio.warp import transform_bounds

import pystac_client
import planetary_computer as pc

from tqdm import tqdm

print("Imports OK!")

Imports OK!


In [2]:
# Charger les données GEMStat
gemstat = pd.read_csv('../data/raw/gemstat_eastern_cape.csv')
print(f"Observations GEMStat : {len(gemstat)}")

# Sites uniques
sites = gemstat[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
print(f"Sites uniques : {len(sites)}")

Observations GEMStat : 1415
Sites uniques : 25


## 1. Extraction DEM (par site)

In [3]:
# =============================================================================
# DEM - Copernicus GLO-30
# =============================================================================

def extract_dem(catalog, lat, lon, buffer_deg=0.005):
    results = {'elevation': np.nan, 'slope': np.nan, 'aspect': np.nan}
    try:
        bbox = [lon - buffer_deg, lat - buffer_deg, lon + buffer_deg, lat + buffer_deg]
        search = catalog.search(collections=["cop-dem-glo-30"], bbox=bbox)
        items = list(search.items())
        if len(items) == 0:
            return results
        item = items[0]
        signed_asset = pc.sign(item.assets["data"])
        with rasterio.open(signed_asset.href) as src:
            dst_crs = src.crs
            transformed_bbox = transform_bounds(CRS.from_epsg(4326), dst_crs, *bbox)
            window = from_bounds(*transformed_bbox, src.transform)
            data = src.read(1, window=window)
            if data.size == 0:
                return results
            valid_data = data[data > -1000]
            if valid_data.size == 0:
                return results
            results['elevation'] = float(np.mean(valid_data))
            # Slope
            if data.size >= 9:
                dy, dx = np.gradient(data, 30)
                slope_rad = np.arctan(np.sqrt(dx**2 + dy**2))
                results['slope'] = float(np.nanmean(np.degrees(slope_rad)))
                # Aspect
                aspect_rad = np.arctan2(-dx, dy)
                aspect_deg = (np.degrees(aspect_rad) + 360) % 360
                results['aspect'] = float(np.nanmean(aspect_deg))
    except:
        pass
    return results

# Connexion
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace,
)
print("Connexion Planetary Computer OK!")

# Extraction
dem_results = []
for idx, row in tqdm(sites.iterrows(), total=len(sites), desc="DEM"):
    result = extract_dem(catalog, row['Latitude'], row['Longitude'])
    result['Latitude'] = row['Latitude']
    result['Longitude'] = row['Longitude']
    dem_results.append(result)

dem_df = pd.DataFrame(dem_results)
print(f"\nDEM extrait pour {len(dem_df)} sites")
display(dem_df.head())

Connexion Planetary Computer OK!


DEM: 100%|██████████| 25/25 [00:16<00:00,  1.56it/s]


DEM extrait pour 25 sites


,elevation,slope,aspect,Latitude,Longitude
0,188.284271,13.843012,159.349808,-32.515278,28.015556
1,61.807449,9.831351,160.492462,-33.237780,26.994720
2,52.606819,12.228851,137.615143,-33.185361,27.390750
3,193.071960,10.501728,211.462509,-32.991639,27.640028
4,218.560760,10.277413,146.195862,-32.802778,27.856389


## 2. Extraction WorldCover (par site)

In [4]:
# =============================================================================
# ESA WorldCover
# =============================================================================

WORLDCOVER_CLASSES = {
    10: 'lc_tree', 20: 'lc_shrubland', 30: 'lc_grassland', 40: 'lc_cropland',
    50: 'lc_builtup', 60: 'lc_bare', 80: 'lc_water', 90: 'lc_wetland'
}

def extract_worldcover(catalog, lat, lon, buffer_meters=500):
    results = {name: 0.0 for name in WORLDCOVER_CLASSES.values()}
    try:
        delta_lat = buffer_meters / 111000
        delta_lon = buffer_meters / (111000 * np.cos(np.radians(lat)))
        bbox = [lon - delta_lon, lat - delta_lat, lon + delta_lon, lat + delta_lat]
        search = catalog.search(collections=["esa-worldcover"], bbox=bbox)
        items = list(search.items())
        if len(items) == 0:
            return results
        item = items[0]
        signed_asset = pc.sign(item.assets["map"])
        with rasterio.open(signed_asset.href) as src:
            dst_crs = src.crs
            transformed_bbox = transform_bounds(CRS.from_epsg(4326), dst_crs, *bbox)
            window = from_bounds(*transformed_bbox, src.transform)
            data = src.read(1, window=window)
            if data.size == 0:
                return results
            total_pixels = data.size
            unique, counts = np.unique(data, return_counts=True)
            for class_code, class_name in WORLDCOVER_CLASSES.items():
                if class_code in unique:
                    idx = np.where(unique == class_code)[0][0]
                    results[class_name] = (counts[idx] / total_pixels) * 100
    except:
        pass
    return results

# Extraction
wc_results = []
for idx, row in tqdm(sites.iterrows(), total=len(sites), desc="WorldCover"):
    result = extract_worldcover(catalog, row['Latitude'], row['Longitude'])
    result['Latitude'] = row['Latitude']
    result['Longitude'] = row['Longitude']
    wc_results.append(result)

wc_df = pd.DataFrame(wc_results)
print(f"\nWorldCover extrait pour {len(wc_df)} sites")
display(wc_df.head())

WorldCover: 100%|██████████| 25/25 [00:07<00:00,  3.54it/s]


WorldCover extrait pour 25 sites


,lc_tree,lc_shrubland,lc_grassland,lc_cropland,lc_builtup,lc_bare,lc_water,lc_wetland,Latitude,Longitude
0,47.526042,29.774306,9.476273,4.810475,0.014468,0.028935,7.581019,0.788484,-32.515278,28.015556
1,8.333333,50.000000,34.259259,0.000000,1.851852,0.000000,0.000000,5.555556,-33.237780,26.994720
2,46.518806,31.057996,18.224232,0.014355,2.067183,0.617284,1.091013,0.409130,-33.185361,27.390750
3,61.663796,17.829457,17.915590,2.239449,0.100488,0.000000,0.251220,0.000000,-32.991639,27.640028
4,85.773758,2.318404,10.752225,1.091013,0.000000,0.000000,0.064599,0.000000,-32.802778,27.856389


## 3. Extraction SoilGrids (par site)

In [5]:
# =============================================================================
# SoilGrids (API ISRIC)
# =============================================================================

SOILGRIDS_API_URL = "https://rest.isric.org/soilgrids/v2.0/properties/query"
SOILGRIDS_PROPERTIES = {
    'phh2o': 'soil_ph', 'clay': 'soil_clay', 'sand': 'soil_sand',
    'soc': 'soil_soc', 'cec': 'soil_cec', 'nitrogen': 'soil_nitrogen'
}

def extract_soilgrids(lat, lon):
    results = {name: np.nan for name in SOILGRIDS_PROPERTIES.values()}
    try:
        params = {
            'lon': lon, 'lat': lat,
            'property': list(SOILGRIDS_PROPERTIES.keys()),
            'depth': ['0-5cm'], 'value': ['mean']
        }
        response = requests.get(SOILGRIDS_API_URL, params=params, timeout=30)
        if response.status_code != 200:
            return results
        data = response.json()
        if 'properties' in data and 'layers' in data['properties']:
            for layer in data['properties']['layers']:
                prop_name = layer['name']
                if prop_name in SOILGRIDS_PROPERTIES:
                    feature_name = SOILGRIDS_PROPERTIES[prop_name]
                    for depth_data in layer['depths']:
                        if depth_data['label'] == '0-5cm':
                            value = depth_data['values'].get('mean')
                            if value is not None:
                                results[feature_name] = float(value)
                            break
    except:
        pass
    return results

# Extraction
soil_results = []
for idx, row in tqdm(sites.iterrows(), total=len(sites), desc="SoilGrids"):
    result = extract_soilgrids(row['Latitude'], row['Longitude'])
    result['Latitude'] = row['Latitude']
    result['Longitude'] = row['Longitude']
    soil_results.append(result)
    time.sleep(0.5)  # Rate limiting

soil_df = pd.DataFrame(soil_results)
print(f"\nSoilGrids extrait pour {len(soil_df)} sites")
display(soil_df.head())

SoilGrids: 100%|██████████| 25/25 [00:31<00:00,  1.28s/it]


SoilGrids extrait pour 25 sites


,soil_ph,soil_clay,soil_sand,soil_soc,soil_cec,soil_nitrogen,Latitude,Longitude
0,61.0,274.0,413.0,350.0,203.0,329.0,-32.515278,28.015556
1,65.0,248.0,531.0,309.0,230.0,300.0,-33.237780,26.994720
2,63.0,227.0,526.0,415.0,246.0,320.0,-33.185361,27.390750
3,60.0,234.0,478.0,374.0,200.0,319.0,-32.991639,27.640028
4,58.0,209.0,445.0,349.0,193.0,350.0,-32.802778,27.856389


## 4. Fusionner les features par site

In [6]:
# Fusionner toutes les features par site
site_features = sites.copy()

# Merge DEM
site_features = site_features.merge(dem_df, on=['Latitude', 'Longitude'], how='left')

# Merge WorldCover
site_features = site_features.merge(wc_df, on=['Latitude', 'Longitude'], how='left')

# Merge SoilGrids
site_features = site_features.merge(soil_df, on=['Latitude', 'Longitude'], how='left')

print(f"Features par site : {site_features.shape}")
display(site_features.head())

Features par site : (25, 19)


,Latitude,Longitude,elevation,slope,aspect,lc_tree,lc_shrubland,lc_grassland,lc_cropland,lc_builtup,lc_bare,lc_water,lc_wetland,soil_ph,soil_clay,soil_sand,soil_soc,soil_cec,soil_nitrogen
0,-32.515278,28.015556,188.284271,13.843012,159.349808,47.526042,29.774306,9.476273,4.810475,0.014468,0.028935,7.581019,0.788484,61.0,274.0,413.0,350.0,203.0,329.0
1,-33.237780,26.994720,61.807449,9.831351,160.492462,8.333333,50.000000,34.259259,0.000000,1.851852,0.000000,0.000000,5.555556,65.0,248.0,531.0,309.0,230.0,300.0
2,-33.185361,27.390750,52.606819,12.228851,137.615143,46.518806,31.057996,18.224232,0.014355,2.067183,0.617284,1.091013,0.409130,63.0,227.0,526.0,415.0,246.0,320.0
3,-32.991639,27.640028,193.071960,10.501728,211.462509,61.663796,17.829457,17.915590,2.239449,0.100488,0.000000,0.251220,0.000000,60.0,234.0,478.0,374.0,200.0,319.0
4,-32.802778,27.856389,218.560760,10.277413,146.195862,85.773758,2.318404,10.752225,1.091013,0.000000,0.000000,0.064599,0.000000,58.0,209.0,445.0,349.0,193.0,350.0


## 5. Extraction TerraClimate (par observation)

Les features climatiques dépendent de la date, donc on doit les extraire pour chaque observation.

In [7]:
# =============================================================================
# TERRACLIMATE - Extraction simplifiée (variables de base)
# =============================================================================

# Pour gagner du temps, on va extraire seulement les variables principales
# Les features temporelles (lags, cumuls) seront calculées après

TERRACLIMATE_VARS = ['ppt', 'tmax', 'tmin', 'soil', 'def', 'vpd', 'aet', 'pet']

def extract_terraclimate_simple(catalog, lat, lon, date_str):
    """Extraction simplifiée de TerraClimate pour une date."""
    results = {var: np.nan for var in TERRACLIMATE_VARS}
    
    try:
        # Convertir la date
        date = pd.to_datetime(date_str, format='%d-%m-%Y')
        year = date.year
        month = date.month
        
        # Bbox
        delta = 0.01
        bbox = [lon - delta, lat - delta, lon + delta, lat + delta]
        
        # Recherche
        time_range = f"{year}-{month:02d}-01/{year}-{month:02d}-28"
        search = catalog.search(
            collections=["terraclimate"],
            bbox=bbox,
            datetime=time_range
        )
        items = list(search.items())
        
        if len(items) == 0:
            return results
        
        item = items[0]
        
        for var in TERRACLIMATE_VARS:
            if var in item.assets:
                signed_asset = pc.sign(item.assets[var])
                with rasterio.open(signed_asset.href) as src:
                    dst_crs = src.crs
                    transformed_bbox = transform_bounds(CRS.from_epsg(4326), dst_crs, *bbox)
                    window = from_bounds(*transformed_bbox, src.transform)
                    data = src.read(1, window=window)
                    if data.size > 0:
                        valid = data[data != src.nodata] if src.nodata else data
                        if valid.size > 0:
                            results[var] = float(np.mean(valid))
    except:
        pass
    
    return results

print("TerraClimate - Extraction pour les observations GEMStat")
print(f"Observations à traiter : {len(gemstat)}")
print("\n⚠️ Cette extraction peut prendre 15-30 minutes...")

TerraClimate - Extraction pour les observations GEMStat
Observations à traiter : 1415

⚠️ Cette extraction peut prendre 15-30 minutes...


In [8]:
# Extraction TerraClimate
terra_results = []

for idx, row in tqdm(gemstat.iterrows(), total=len(gemstat), desc="TerraClimate"):
    result = extract_terraclimate_simple(
        catalog, row['Latitude'], row['Longitude'], row['Sample Date']
    )
    result['Latitude'] = row['Latitude']
    result['Longitude'] = row['Longitude']
    result['Sample Date'] = row['Sample Date']
    terra_results.append(result)

terra_df = pd.DataFrame(terra_results)
print(f"\nTerraClimate extrait pour {len(terra_df)} observations")

TerraClimate: 100%|██████████| 1415/1415 [01:32<00:00, 15.28it/s]


TerraClimate extrait pour 1415 observations


## 6. Fusion finale

In [9]:
# =============================================================================
# FUSION FINALE
# =============================================================================

# Partir des données GEMStat originales
gemstat_features = gemstat.copy()

# Ajouter les features par site (DEM, WorldCover, SoilGrids)
gemstat_features = gemstat_features.merge(
    site_features,
    on=['Latitude', 'Longitude'],
    how='left'
)

# Ajouter les features TerraClimate (par observation)
gemstat_features = gemstat_features.merge(
    terra_df,
    on=['Latitude', 'Longitude', 'Sample Date'],
    how='left'
)

print(f"Dataset GEMStat avec features : {gemstat_features.shape}")
print(f"Colonnes : {gemstat_features.columns.tolist()}")

Dataset GEMStat avec features : (1415, 31)
Colonnes : ['Latitude', 'Longitude', 'Sample Date', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus', 'elevation', 'slope', 'aspect', 'lc_tree', 'lc_shrubland', 'lc_grassland', 'lc_cropland', 'lc_builtup', 'lc_bare', 'lc_water', 'lc_wetland', 'soil_ph', 'soil_clay', 'soil_sand', 'soil_soc', 'soil_cec', 'soil_nitrogen', 'ppt', 'tmax', 'tmin', 'soil', 'def', 'vpd', 'aet', 'pet']


In [10]:
# Sauvegarder
gemstat_features.to_csv('../data/processed/gemstat_features.csv', index=False)
print("✅ Sauvegardé : gemstat_features.csv")

# Statistiques
print(f"\nStatistiques :")
print(f"  Observations : {len(gemstat_features)}")
print(f"  Features : {len(gemstat_features.columns) - 6}")  # -6 pour lat, lon, date, 3 targets
print(f"  Valeurs manquantes :")
missing = gemstat_features.isnull().sum()
for col in missing[missing > 0].index[:10]:
    print(f"    {col}: {missing[col]} ({missing[col]/len(gemstat_features)*100:.1f}%)")

✅ Sauvegardé : gemstat_features.csv

Statistiques :
  Observations : 1415
  Features : 25
  Valeurs manquantes :
    ppt: 1415 (100.0%)
    tmax: 1415 (100.0%)
    tmin: 1415 (100.0%)
    soil: 1415 (100.0%)
    def: 1415 (100.0%)
    vpd: 1415 (100.0%)
    aet: 1415 (100.0%)
    pet: 1415 (100.0%)


## Résumé

**Fichier créé :**
- `gemstat_features.csv` : Données GEMStat avec toutes les features

**Note :** L'extraction Landsat n'est pas incluse car elle prendrait plusieurs heures. 
On peut d'abord tester le modèle avec les features actuelles (DEM, WorldCover, SoilGrids, TerraClimate).

**Prochaine étape :**
- Fusionner avec le training original (qui a les features Landsat)
- Réentraîner le modèle